**<center><h1>Introduction</h1></center>**

Application Insights is an application performance management service in Microsoft Azure that enables the capture, storage, and analysis of telemetry data from applications.

<img src="images/10-01-app-sights.png" />

You can use Application Insights to monitor telemetry from many kinds of application, including applications that are not running in Azure. All that's required is a low-overhead instrumentation package to capture and send the telemetry data to Application Insights. The necessary package is already included in Azure Machine Learning Web services, so you can use it to capture and review telemetry from models published with Azure Machine Learning.

**<h2>Learning objectives</h2>**

In this module, you will learn how to:

- Enable Application Insights monitoring for an Azure Machine Learning web service.
- Capture and view model telemetry.


<hr>

**<center><h1>Enable Application Insights</h1></center>**

To log telemetry in application insights from an Azure machine learning service, you must have an Application Insights resource associated with your Azure Machine Learning workspace, and you must configure your service to use it for telemetry logging.




**<h2>Associate Application Insights with a workspace</h2>**

When you create an Azure Machine Learning workspace, you can select an Azure Application Insights resource to associate with it. If you do not select an existing Application Insights resource, a new one is created in the same resource group as your workspace.

You can determine the Application Insights resource associated with your workspace by viewing the Overview page of the workspace blade in the Azure portal, or by using the get_details() method of a Workspace object as shown in the following code example:
```
from azureml.core import Workspace

ws = Workspace.from_config()
ws.get_details()['applicationInsights']
```


**<h2>Enable Application Insights for a service</h2>**
When deploying a new real-time service, you can enable Application Insights in the deployment configuration for the service, as shown in this example:
```
dep_config = AciWebservice.deploy_configuration(cpu_cores = 1,
                                                memory_gb = 1,
                                                enable_app_insights=True)
```
If you want to enable Application Insights for a service that is already deployed, you can modify the deployment configuration for Azure Kubernetes Service (AKS) based services in the Azure portal. Alternatively, you can update any web service by using the Azure Machine Learning SDK, like this:
```
service = ws.webservices['my-svc']
service.update(enable_app_insights=True)
```





<hr>

**<center><h1>Capture and view telemetry</h1></center>**

Application Insights automatically captures any information written to the standard output and error logs, and provides a query capability to view data in these logs.


**<h2>Write log data</h2>**

To capture telemetry data for Application insights, you can write any values to the standard output log in the scoring script for your service by using a `print` statement, as shown in the following example:
```
def init():
    global model
    model = joblib.load(Model.get_model_path('my_model'))
def run(raw_data):
    data = json.loads(raw_data)['data']
    predictions = model.predict(data)
    log_txt = 'Data:' + str(data) + ' - Predictions:' + str(predictions)
    print(log_txt)
    return predictions.tolist()
```
Azure Machine Learning creates a custom dimension in the Application Insights data model for the output you write.


**<h2>Query logs in Application Insights</h2>**

To analyze captured log data, you can use the Log Analytics query interface for Application Insights in the Azure portal. This interface supports a SQL-like query syntax that you can use to extract fields from logged data, including custom dimensions created by your Azure Machine Learning service.

For example, the following query returns the **timestamp** and **customDimensions.Content** fields from log traces that have a **message** field value of STDOUT (indicating the data is in the standard output log) and a **customDimensions.["Service Name"]** field value of my-svc:

```
# SQL
traces
|where message == "STDOUT"
  and customDimensions.["Service Name"] = "my-svc"
| project  timestamp, customDimensions.Content
```
This query returns the logged data as a table:

<img src="images/image4.png" />




<hr>

**<center><h1>Exercise - Monitor a model</h1></center>**


Now it's your chance to monitor a model that is deployed as an Azure Machine Learning real-time service.

In this exercise, you will:

- Configure Application Insights for a deployed service.
- Capture and view model telemetry.


**<h2>Instructions</h2>**

Follow these instructions to complete the exercise.

1. If you do not already have an Azure subscription, sign up for a free trial at https://azure.microsoft.com.
2. View the exercise repo at https://aka.ms/mslearn-dp100.
3. If you have not already done so, complete the Create an Azure Machine Learning workspace exercise to provision an Azure Machine Learning workspace, create a compute instance, and clone the required files.
4. Complete the **Monitor a model** exercise.



<hr>

**<center><h1>Knowledge check</h1></center>**

1. You have deployed a model as a real-time inferencing service in an Azure Kubernetes Service (AKS) cluster. What must you do to capture and analyze telemetry for this service?

- Redeploy the model as an ACI service.

- Enable application insights.

- Move the AKS cluster to the same region as the Azure Machine Learning workspace.

2. You want to include custom information in the telemetry for your inferencing service, and analyze it using Application Insights. What must you do in your service's entry script?

- Use the Run.log method to log the custom metrics.

- Save the custom metrics in the ./outputs folder.

- Use a print statement to write the metrics in the STDOUT log.


<hr>

**<center><h1>Summary</h1></center>**

In this module, you learned how to:

- Enable Application Insights monitoring for an Azure Machine Learning web service.
- Capture and view model telemetry.


To learn more about monitoring models with Azure Machine Learning, see [Monitor and collect data from ML web service endpoints](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-enable-app-insights) in the Azure Machine Learning documentation.


<hr>